## RayPipeline with detached component actors 

When you run `RayPipeline` under the hood it creates [ray actors](https://docs.ray.io/en/latest/ray-core/actors.html) for each component in the pipeline (`ComponentActor` instances). 
`ComponentActor` exposes `run_component` method which can be remotely executed by `RayPipelineProcessor` - another actor which orchestrates pipeline execution logic (e.g. runs components according to the order defined by pipeline connections). Lifetime of Haystack component instance is same as actor's. Please notice each Haystack component might have its own logic which could be not idempotent - we can not control it.

In case Haystack component requires `warm_up` - it will perform it during component's first run (first time `run_component` is called). Sometimes `warm_up` might be an expensive operation and ideally we would like to avoid it when we call `run_component` again. Internally ComponentActor will store a flag in case component has already been warmed up so that next time it runs we skip this step.

Generally, whenever you run pipeline with `pipeline.run` the following happens:

1. Pipeline is validated for correctness, along with given inputs if any
2. `RayPipelineProcessor` actor is created
3. `ComponentActor` is created for each component in pipeline
4. `RayPipelineProcessor` starts pipeline execution
5. When pipeline execution finishes all actors will be destroyed by default. Next time you call `pipeline.run` steps will be repeated, so **actor instances will be re-created**. 

In cases you would like to keep component actors alive between sequential pipeline runs that can be achieved with [Named Actors](https://docs.ray.io/en/latest/ray-core/actors/named-actors.html) and "detached" [Actor Lifetimes](https://docs.ray.io/en/latest/ray-core/actors/named-actors.html#actor-lifetimes). `ray-haystack` allows you to control actor names and lifetimes with `RayPipelineSettings`. So in order to reuse actor instances between pipeline runs we could configure actors as "detached". Detached actors are not destroyed until manually killed with `ray.kill` or cluster shutdown.

Lets explore "detached" named component actors in this notebook.

### Install Dependencies

In [ ]:
%%bash

pip install "ray[default]" # Ray library with dashboard included
pip install ray-haystack
pip install "datasets>=2.6.1" # Required by pipeline
pip install "sentence-transformers>=3.0.0" # Required by pipeline

### Running a pipeline with default settings

Setup required env variables

In [1]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

Start local Ray cluster

In [ ]:
import ray

# Shutdown cluster if any is running at the moment
if ray.is_initialized():
    ray.shutdown()

ray.init()

Prepare DocumentStore by writing Documents with embeddings.

`RayInMemoryDocumentStore` is actually runs within an actor behind the scenes so we could share a single instance of it in Ray cluster

In [ ]:
from datasets import load_dataset
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.dataclasses import Document

from ray_haystack.utils import RayInMemoryDocumentStore

document_store = RayInMemoryDocumentStore()

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Create RayPipeline for basic RAG

In [ ]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.embedders import (
    SentenceTransformersTextEmbedder,
)
from haystack.components.generators import OpenAIGenerator

from ray_haystack.ray_pipeline import RayPipeline
from ray_haystack.utils import RayInMemoryEmbeddingRetriever

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = RayInMemoryEmbeddingRetriever(document_store) # document_store is created above
generator = OpenAIGenerator(model="gpt-3.5-turbo")

prompt_builder = PromptBuilder(template=template)

basic_rag_pipeline = RayPipeline()

basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

Run pipeline with default settings

Note: While pipeline is running navigate to http://127.0.0.1:8265/#/actors to see all actors being created for component execution, and then destroyed when pipeline finishes 

In [ ]:
question = "What does Rhodes Statue look like?"

basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Lets check if any actor is still available after pipeline finishes execution:

In [ ]:
# This will result in exception as actor does not exist (in DEAD state)

ray.get_actor(name="llm")

Note: There could be some delay until actor is actually destroyed by Ray (e.g. have gone out of scope in Python). It could be a problem if you immediately create actor with same name (with "non_detached" lifetime) - it will result in error as no actors with same name can exist in same namespace. 

### Pipeline with detached component actors

`RayPipeline` can be configured with particular [actor options](https://docs.ray.io/en/latest/ray-core/api/doc/ray.actor.ActorClass.options.html) for all components within the pipeline.

So it should be possible to specify the lifetime of actors in case we don't want those to be destroyed when pipeline finishes.

Lets create same pipeline using settings to detach component actors and reuse same actor instances next time pipeline runs

In [7]:
from ray_haystack import RayPipelineSettings

ray_settings: RayPipelineSettings = {
    "common": {
        "actor_options": {
            "lifetime": "detached", # Don't kill the actor after pipeline finishes
            "get_if_exists": True, # When creating actor get one that already exists (with same name)
        }
    }
}

Run pipeline with common component actor settings

In [ ]:
question = "Why did people build Great Pyramid of Giza?"

basic_rag_pipeline.run(
    {"text_embedder": {"text": question}, "prompt_builder": {"question": question}}, ray_settings=ray_settings
)

Get actor by name, it should be still available in detached state

In [ ]:
ray.get_actor(name="llm")

Lets run pipeline once again, it should reuse previously created component actors

In [ ]:
question = "How did Colossus of Rhodes collapse?"

basic_rag_pipeline.run(
    {"text_embedder": {"text": question}, "prompt_builder": {"question": question}}, ray_settings=ray_settings
)

The above pipeline execution should reuse component actors, potentially improving performance as there is no additional overhead. It should be quite useful when component needs to "warm up" and we would like to avoid repeating this step each time `run` method is called.

Depending on your use case you can setup `RayPipeline` with various actor options during runtime.

#### Terminate detached actors

According to [documentation](https://docs.ray.io/en/latest/ray-core/actors/terminating-actors.html) detached actors must be manually destroyed. 

So you will need a way to clean up actors so that those do not get stuck running and consuming resources once you no longer need working/running a pipeline.

One way to do it is to get actor handle and then use `ray.kill` util to terminate the actor, e.g.

```py
actor = ray.get_actor("component_name") # Assuming you did not change default name for the actor
ray.kill(actor)
```

Below is a more generic version of actor "cleanup" logic:

In [ ]:
for component_name in basic_rag_pipeline.get_component_names():
    ray.kill(ray.get_actor(component_name))

Note: If actor can not be found by name `ray.get_actor` will raise an error (ValueError). 

Please also consider cases when you manually provide name for a component actor - in such case you should find the actor by its custom name - not by component name

**Alternatively there is an easy option to kill detached actors:**

In [ ]:
basic_rag_pipeline.cleanup() # kill actors belonging to the pipeline which were created as detached

### Shutdown Ray cluster

In [ ]:
ray.shutdown()